In [ ]:
#!pip install xgboost

In [ ]:
import os
import pandas as pd
import numpy as np

import psycopg2
import sqlalchemy 
from sqlalchemy import create_engine
from sqlalchemy import text

from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

from datetime import timedelta

In [ ]:
with open("credentials/bbdd.json") as f:
    bbdd = json.load(f)

host=bbdd["host"]
database=bbdd["database"]
user=bbdd["user"]
password=bbdd["password"]

In [ ]:
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}/{database}")

In [ ]:
#Bloque de test - cierra automáticamente la conexión
with engine.connect() as conn:
    rows = conn.execute(text("SELECT 1")).all()
    df = pd.read_sql_table(table_name = "demanda_electrica", con = conn)
    print(df.head())

In [ ]:
connection = engine.connect()

In [ ]:
df = pd.read_sql_table(table_name = "demanda_electrica", con = connection)

In [ ]:
df

In [ ]:
connection.close()

In [ ]:
engine.dispose()

In [22]:
df_backup = df.copy()

#df = df_backup

In [18]:
df_clean = df.copy()

,value,datetime,datetime_utc,tz_time,geo_id,geo_name
0,3187298.0,2014-01-01T00:00:00.000+01:00,2013-12-31T23:00:00Z,2013-12-31T23:00:00.000Z,8741,Península
1,3916726.0,2014-01-02T00:00:00.000+01:00,2014-01-01T23:00:00Z,2014-01-01T23:00:00.000Z,8741,Península
2,3957472.0,2014-01-03T00:00:00.000+01:00,2014-01-02T23:00:00Z,2014-01-02T23:00:00.000Z,8741,Península
3,3726719.0,2014-01-04T00:00:00.000+01:00,2014-01-03T23:00:00Z,2014-01-03T23:00:00.000Z,8741,Península
4,3511386.0,2014-01-05T00:00:00.000+01:00,2014-01-04T23:00:00Z,2014-01-04T23:00:00.000Z,8741,Península


In [31]:
df_clean.isnull().sum()

value           0
datetime        0
datetime_utc    0
tz_time         0
geo_id          0
geo_name        0
dtype: int64

In [33]:
df_clean["geo_name"].value_counts(dropna=False)

geo_name
Península    4276
Name: count, dtype: int64

In [30]:
df_clean["geo_id"].value_counts(dropna=False)

geo_id
8741    4276
Name: count, dtype: int64

In [41]:
df_clean["value"]

0       3187298.0
1       3916726.0
2       3957472.0
3       3726719.0
4       3511386.0
          ...    
4271    7532009.0
4272    7719284.0
4273    6931088.0
4274    6490902.0
4275    3386155.0
Name: value, Length: 4276, dtype: float64

In [42]:
df_clean["datetime"]

0       2014-01-01T00:00:00.000+01:00
1       2014-01-02T00:00:00.000+01:00
2       2014-01-03T00:00:00.000+01:00
3       2014-01-04T00:00:00.000+01:00
4       2014-01-05T00:00:00.000+01:00
                    ...              
4271    2025-09-11T00:00:00.000+02:00
4272    2025-09-12T00:00:00.000+02:00
4273    2025-09-13T00:00:00.000+02:00
4274    2025-09-14T00:00:00.000+02:00
4275    2025-09-15T00:00:00.000+02:00
Name: datetime, Length: 4276, dtype: object

In [43]:
df_clean["datetime_utc"]

0       2013-12-31T23:00:00Z
1       2014-01-01T23:00:00Z
2       2014-01-02T23:00:00Z
3       2014-01-03T23:00:00Z
4       2014-01-04T23:00:00Z
                ...         
4271    2025-09-10T22:00:00Z
4272    2025-09-11T22:00:00Z
4273    2025-09-12T22:00:00Z
4274    2025-09-13T22:00:00Z
4275    2025-09-14T22:00:00Z
Name: datetime_utc, Length: 4276, dtype: object

In [45]:
df_clean["converted_datetime_utc_date"] = pd.to_datetime(df_clean["datetime_utc"], utc=True)

df_clean["converted_datetime_utc_date"]

0      2013-12-31 23:00:00+00:00
1      2014-01-01 23:00:00+00:00
2      2014-01-02 23:00:00+00:00
3      2014-01-03 23:00:00+00:00
4      2014-01-04 23:00:00+00:00
                  ...           
4271   2025-09-10 22:00:00+00:00
4272   2025-09-11 22:00:00+00:00
4273   2025-09-12 22:00:00+00:00
4274   2025-09-13 22:00:00+00:00
4275   2025-09-14 22:00:00+00:00
Name: converted_datetime_utc_date, Length: 4276, dtype: datetime64[ns, UTC]

In [51]:
df_clean_backup = df_clean.copy()

df_clean_backup = df_clean_backup.copy()

In [52]:
df_clean = df_clean.drop(columns = ["datetime", "datetime_utc", "tz_time", "geo_id", "geo_name"])

df_clean

,value,converted_datetime_utc_date
0,3187298.0,2013-12-31 23:00:00+00:00
1,3916726.0,2014-01-01 23:00:00+00:00
2,3957472.0,2014-01-02 23:00:00+00:00
3,3726719.0,2014-01-03 23:00:00+00:00
4,3511386.0,2014-01-04 23:00:00+00:00
...,...,...
4271,7532009.0,2025-09-10 22:00:00+00:00
4272,7719284.0,2025-09-11 22:00:00+00:00
4273,6931088.0,2025-09-12 22:00:00+00:00
4274,6490902.0,2025-09-13 22:00:00+00:00


In [59]:
corte  = df_clean["converted_datetime_utc_date"].max() - pd.Timedelta(days=60)

test_df = df_clean[df_clean["converted_datetime_utc_date"] >= corte]
train_df = df_clean[df_clean["converted_datetime_utc_date"] <  corte]

print(f"Train: \n Fecha min: {train_df["converted_datetime_utc_date"].min()} \n Fecha max: {train_df["converted_datetime_utc_date"].max()} \n Len: {len(train_df)}")
print(f"Test: \n Fecha min: {test_df["converted_datetime_utc_date"].min()} \n Fecha max: {test_df["converted_datetime_utc_date"].max()} \n Len: {len(test_df)}")

Train: 
 Fecha min: 2013-12-31 23:00:00+00:00 
 Fecha max: 2025-07-15 22:00:00+00:00 
 Len: 4215
Test: 
 Fecha min: 2025-07-16 22:00:00+00:00 
 Fecha max: 2025-09-14 22:00:00+00:00 
 Len: 61
